In [6]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable, Derive
from biogeme.models import logit, loglogit, piecewiseFormula, nested
from biogeme.models import lognested
from biogeme.results import bioResults, pickle

In [7]:
df = pd.read_table("new_lpmc02.dat", index_col=0)
database = db.Database("LPMC",df)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

In [8]:
# Model

  
# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public = DefineVariable('cost_public', cost_transit ,database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 *\
                    car_ownership +\
                    Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1, 2: 1, 3: 1, 4: 1}


#Definitions of nests
N_SM = N_SM, [1, 2]
N_MOTOR = N_MOTOR, [3, 4]

nests = N_SM, N_MOTOR

In [9]:
sum_weights = database.data['Weights'].sum()
S = database.getSampleSize()
sample_normalized_weight = Weights * S / sum_weights

In [10]:
# market share prediction 

prob_walking = nested(V, av, nests, 1)
prob_cycling = nested(V, av, nests, 2)
prob_public = nested(V, av, nests, 3)
prob_car = nested(V, av, nests, 4)

# agg cost elasticities
direct_elas_public_cost = Derive(prob_public, "cost_public") * cost_public / prob_public
direct_elas_car_cost = Derive(prob_car, "cost_driving") * cost_driving / prob_car



simulate = {'Prob. walking': prob_walking,
            'Prob. cycling': prob_cycling,
            'Prob. public': prob_public,
            'Prob. car': prob_car,
            'Weighted prob. walking': sample_normalized_weight * prob_walking,
            'Weighted prob. cycling': sample_normalized_weight * prob_cycling,
            'Weighted prob. public': sample_normalized_weight * prob_public,
            'Weighted prob. car': sample_normalized_weight * prob_car,
            'Revenue public': prob_public * cost_public,
            'Revenue driving': prob_car * cost_driving,
            'direct_elas_public_cost': direct_elas_public_cost,
            'direct_elas_car_cost': direct_elas_car_cost
           }

output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_nested_lpmc_sm_motor")
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)
    
# # delete previously saved html and pickle
# for file in glob.glob(f"{filepath}*"):
#     os.remove(file)

biogeme  = bio.BIOGEME(database, simulate)
#biogeme.modelName = filepath

betas = biogeme.freeBetaNames
results = bioResults(pickleFile=f"{filepath}.pickle")

beta_values = results.getBetaValues()

In [81]:
simulated_values = biogeme.simulate(beta_values)

marketShare_walking = 100 * simulated_values['Weighted prob. walking'].mean()
marketShare_cycling = 100 * simulated_values['Weighted prob. cycling'].mean()
marketShare_public = 100 * simulated_values['Weighted prob. public'].mean()
marketShare_car = 100 * simulated_values['Weighted prob. car'].mean()

normalization_public = simulated_values["Weighted prob. public"].sum() 
normalization_car = simulated_values["Weighted prob. car"].sum()

agg_elas_public_cost = (simulated_values['Weighted prob. public']*simulated_values['direct_elas_public_cost']/\
                         normalization_public).sum()
agg_elas_car_cost = (simulated_values['Weighted prob. car']*simulated_values['direct_elas_car_cost']/\
                         normalization_car).sum()

print(agg_elas_public_cost, normalization_public)
print(agg_elas_car_cost, normalization_car)

-0.08321740484370775 1821.5732013979386
-0.061471438784222364 2157.042118394166


In [80]:
revenue_public_avg = simulated_values["Revenue public"].mean()
revenue_public_total = simulated_values["Revenue public"].sum()

revenue_public_total

3577.1343927456273

In [74]:
# conf interval
b = results.getBetasForSensitivityAnalysis(betas, size=100)
left, right = biogeme.confidenceIntervals(b, 0.9)

In [75]:
pd.DataFrame(left).head(10)

,Prob. walking,Prob. cycling,Prob. public,Prob. car,Weighted prob. walking,Weighted prob. cycling,Weighted prob. public,Weighted prob. car,Revenue public,Revenue driving,VOT public,VOT car,Weighted VOT public,Weighted VOT car,direct_elas_public_cost,direct_elas_car_cost
0,0.169,0.0503,0.464,0.144,0.181,0.0539,0.497,0.154,0,0.0577,13,49.7,13.9,53.2,0,-0.0536
1,7.71e-05,0.0105,0.922,0.0253,8.25e-05,0.0112,0.987,0.0271,3.78,0.0563,14.2,19,15.2,20.3,-0.0437,-0.377
2,0.0268,0.0123,0.0293,0.864,0.0287,0.0132,0.0313,0.925,0.0439,0.372,20.2,60.5,21.6,64.8,-0.254,-0.00673
3,0.0393,0.00589,0.0697,0.765,0.042,0.00631,0.0746,0.819,0,0.406,8.06,38,8.63,40.7,0,-0.0152
4,0.577,0.0139,0.091,0.193,0.618,0.0149,0.0974,0.207,0,0.0425,19.2,85.3,20.5,91.3,0,-0.0249
5,3.28e-07,0.00544,0.26,0.57,3.51e-07,0.00582,0.278,0.61,1.98,2.35,11.6,16.8,12.4,18,-0.932,-0.249
6,0.139,0.0325,0.185,0.511,0.149,0.0347,0.198,0.546,0.278,0.199,21.6,62,23.2,66.3,-0.204,-0.0266
7,0.2,0.00767,0.034,0.581,0.214,0.00821,0.0364,0.621,0,0.128,18.8,102,20.2,109,0,-0.0106
8,0.000159,0.00928,0.733,0.124,0.00017,0.00993,0.784,0.133,3.15,0.271,10.7,14.7,11.5,15.8,-0.173,-0.314
9,0.168,0.0113,0.043,0.604,0.18,0.0121,0.046,0.647,0,0.145,19.2,90.3,20.5,96.6,0,-0.0111


In [76]:
pd.DataFrame(right).head(10)

,Prob. walking,Prob. cycling,Prob. public,Prob. car,Weighted prob. walking,Weighted prob. cycling,Weighted prob. public,Weighted prob. car,Revenue public,Revenue driving,VOT public,VOT car,Weighted VOT public,Weighted VOT car,direct_elas_public_cost,direct_elas_car_cost
0,0.229,0.104,0.594,0.239,0.245,0.111,0.636,0.256,0,0.0957,29.7,101,31.8,109,0,-0.0329
1,0.00189,0.029,0.96,0.0536,0.00202,0.031,1.03,0.0574,3.94,0.119,25.1,33.4,26.9,35.7,-0.0188,-0.223
2,0.0501,0.0329,0.0582,0.926,0.0536,0.0352,0.0623,0.991,0.0873,0.398,35.6,108,38.1,115,-0.148,-0.00331
3,0.0853,0.0191,0.129,0.875,0.0913,0.0204,0.138,0.937,0,0.464,22.7,85.3,24.3,91.3,0,-0.0073
4,0.651,0.0548,0.145,0.282,0.697,0.0586,0.155,0.302,0,0.062,48.2,172,51.5,184,0,-0.0125
5,0.000173,0.0217,0.408,0.726,0.000185,0.0233,0.437,0.777,3.1,3,20.6,29.2,22.1,31.3,-0.498,-0.164
6,0.184,0.0647,0.268,0.624,0.197,0.0692,0.287,0.668,0.402,0.243,39.8,110,42.6,118,-0.11,-0.015
7,0.319,0.0366,0.069,0.75,0.342,0.0392,0.0739,0.803,0,0.165,48.1,206,51.5,221,0,-0.00484
8,0.00328,0.0269,0.857,0.234,0.00351,0.0288,0.918,0.25,3.69,0.512,19.6,26.5,21,28.4,-0.0733,-0.186
9,0.273,0.0496,0.0841,0.764,0.292,0.0531,0.09,0.818,0,0.183,48.2,182,51.5,195,0,-0.0051


In [77]:
lst_marketShares = [marketShare_walking, marketShare_cycling, marketShare_public, marketShare_car]
temp_names = ["marketShare_walking", "marketShare_cycling", "marketShare_public", "marketShare_car"]

print("Predicted market shares:\n")
for i in range(len(temp_names)):
    l = left[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    r = right[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    print(f"{temp_names[i]}: {lst_marketShares[i]:.2f}% ({l:.2f}%, {r:.2f}%)")

Predicted market shares:

marketShare_walking: 17.59% (15.79%, 20.03%)
marketShare_cycling: 2.84% (2.20%, 5.96%)
marketShare_public: 36.43% (33.67%, 40.37%)
marketShare_car: 43.14% (36.96%, 45.32%)


In [78]:
lst_wtps = [wtp_public, wtp_car]
wtp_names = ["VOT public", "VOT car"]
print("Average (unweighted) values of time:\n")
for i in range(len(lst_wtps)):
    l = left[f"{wtp_names[i]}"].mean()
    r = right[f"{wtp_names[i]}"].mean()
    print(f"{wtp_names[i]}: {lst_wtps[i]:.2f} ({l:.2f}, {r:.2f})")

Average (unweighted) values of time:

VOT public: 39.15 (27.59, 53.61)
VOT car: 95.91 (65.15, 126.62)


In [60]:
df.head()["travel_mode"], df.columns # sanity check

(0    3
 1    3
 2    4
 3    4
 4    1
 Name: travel_mode, dtype: int64,
 Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
        'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
        'travel_year', 'travel_month', 'travel_date', 'day_of_week',
        'start_time', 'age', 'female', 'driving_license', 'car_ownership',
        'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
        'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
        'dur_driving', 'cost_transit', 'cost_driving_fuel',
        'cost_driving_ccharge', 'driving_traffic_percent', 'Weights',
        'cost_public', 'dur_public', 'cost_driving'],
       dtype='object'))

In [29]:
# compare with actual choices
#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

choices = ["walking", "cycling", "public", "driving"]
actual_choices = df.travel_mode.value_counts()/df.shape[0]

print("Actual choices: \n")
for i in range(len(choices)):
    print(f"{choices[i]}: {actual_choices[i+1]*100:.2f}%")

Actual choices: 

walking: 17.38%
cycling: 2.86%
public: 35.58%
driving: 44.18%


In [69]:
# share of users choosing a mode with a higher (unweighted) probability for another mode, 

temp = {"Prob. walking": 1, "Prob. cycling": 2, "Prob. public": 3, "Prob. car": 4}
simulated_probs = simulated_values[["Prob. walking", "Prob. cycling", "Prob. public", "Prob. car"]]
choices_per_n = pd.DataFrame(simulated_probs.idxmax(axis=1), columns=["mode_by_prob"])
choices_per_n = choices_per_n.replace(temp)
df_merged = df.merge(choices_per_n, left_index=True, right_index=True)

deviates = df_merged[df_merged.travel_mode != df_merged.mode_by_prob]

print(f"Share of those choosing a mode despite having a higher prob. for another mode: "
      f"{deviates.shape[0]/df.shape[0] * 100:.2f}%")

# specifically, within each mode
for i in range(len(choices)):
    total = df.travel_mode.value_counts()[i+1]
    anomalies = deviates.travel_mode.value_counts()[i+1]
    print(f"Out of those who choose {choices[i]}, {anomalies/total * 100:.2f}% have a higher prob. of using"
        " another mode")

Share of those choosing a mode despite having a higher prob. for another mode: 28.86%
Out of those who choose walking, 36.25% have a higher prob. of using another mode
Out of those who choose cycling, 100.00% have a higher prob. of using another mode
Out of those who choose public, 31.08% have a higher prob. of using another mode
Out of those who choose driving, 19.56% have a higher prob. of using another mode


In [ ]:
# revenues
